# 🌊 Exploring Copernicus Marine Data in Google Colab

**An interactive notebook for retrieving and exploring Copernicus Marine datasets, created by Náttúrufræðistofnun.**

---

### Overview
This notebook connects to **Copernicus Marine Service** to retrieve and list all available datasets.
Using Python, it fetches dataset descriptions automatically and generates markdown documentation.

The script logs in Copernicus Marine using credentials and queries Copernicus Marine API for metadata.

Find a full list of available products here: https://data.marine.copernicus.eu/products

### How to Use
1. Open this notebook in Google Colab or another cloud platform.
2. Register a Copernicus Marine account here: https://data.marine.copernicus.eu/register
3. Insert valid credentials in section Login to access the data.
4. Run the provided code cells in sequence (do not skip cells) to explore dataset descriptions and available services, download, process and display them.

| Try the code via free cloud platforms: | [![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lmi/Copernicus/blob/master/Marine/Overview/ConnectMarine.ipynb)

---

### Attribution
- **License**: This notebook is published under **CC BY 4.0**.
- **Citation**: "*Credits: Náttúrufræðistofnun*"
- **Author**: Marco Pizzolato
- **Data Sources**: [Copernicus Marine Service](https://marine.copernicus.eu/)
- **Created in**: Python

---

**This notebook provides an automated way to explore and document Copernicus Marine datasets, making ocean data more accessible.**


## Install and Import

In [62]:
!pip install -q copernicusmarine pandas tabulate Pillow mdutils leafmap rioxarray xarray netCDF4 rasterio pyproj localtileserver

In [63]:
import os
import requests
from PIL import Image
from mdutils.mdutils import MdUtils
from mdutils.tools import Html

## Login to Copernicus Marine

Register a Copernicus Marine account here: https://data.marine.copernicus.eu/register

In [ ]:
import copernicusmarine as cm
import getpass

username = input("Enter your Copernicus Marine username: ")
password = getpass.getpass("Enter your Copernicus Marine password: ")

cm.login(username=username, password=password)

## Fetch Catalog

In [66]:
catalog = cm.describe()

Fetching products:  98%|█████████▊| 296/303 [00:22<00:00, 28.48it/s]WARNING:urllib3.connectionpool:Connection pool is full, discarding connection: s3.waw3-1.cloudferro.com. Connection pool size: 10

Fetching products:  99%|█████████▉| 300/303 [00:23<00:00, 22.77it/s]WARNING:urllib3.connectionpool:Connection pool is full, discarding connection: s3.waw3-1.cloudferro.com. Connection pool size: 10

Fetching catalogue 1: 100%|██████████| 2/2 [00:34<00:00, 17.36s/it]


In [67]:
# Inspect the structure of the first product and its datasets
first_product = catalog.products[0]
print(first_product.model_dump())
print(first_product.datasets)

{'title': 'Antarctic Sea Ice Extent from Reanalysis', 'product_id': 'ANTARCTIC_OMI_SI_extent', 'thumbnail_url': 'https://documentation.marine.copernicus.eu/IMG/ANTARCTIC_OMI_SI_extent.png', 'description': '**DEFINITION**\n\nEstimates of Antarctic sea ice extent are obtained from the surface of oceans grid cells that have at least 15% sea ice concentration. These values are cumulated in the entire Southern Hemisphere (excluding ice lakes) and from 1993 up to real time aiming to:\ni) obtain the Antarctic sea ice extent as expressed in millions of km squared (106 km2) to monitor both the large-scale variability and mean state and change.\nii) to monitor the change in sea ice extent as expressed in millions of km squared per decade (106 km2/decade), or in sea ice extent loss/gain since the beginning of the time series as expressed in percent per decade (%/decade; reference period being the first date of the key figure b) dot-dashed trend line, Vaughan et al., 2013)). For the Southern Hemis

## Save catalog as Markdown File

In [70]:
# Initialize the Markdown file
md_file = MdUtils(file_name='Copernicus_Marine_Products_and_Datasets')

# Iterate through products
for product in catalog.products:
    product_title = product.title
    product_id = product.product_id
    description = product.description
    thumbnail_url = product.thumbnail_url

        # Add thumbnail image if available
    if thumbnail_url:
        # Resize the image to 100x100 pixels using HTML
        img_tag = Html.image(path=thumbnail_url, size='300x200')
        md_file.new_paragraph(img_tag)

    # Add an empty line after the image
    md_file.new_paragraph('\n')

    # Add a header for the product
    md_file.new_header(level=1, title=product_title)

    # Add product ID
    md_file.new_paragraph(f'**Product ID:** {product_id}')

    # Add description
    md_file.new_paragraph(f'**Description:** {description}')

    # Add an empty line after the image
    md_file.new_paragraph('\n')

In [71]:
# Create the Markdown file
md_file.create_md_file()

## Display the created overview

In [72]:
from IPython.display import display, Markdown

In [73]:
# Display the generated markdown file at the end of the notebook
n_lines = 100  # how many lines to display
with open("Copernicus_Marine_Products_and_Datasets.md", "r") as file:
    content = "".join([next(file) for _ in range(n_lines)])
display(Markdown(content + "\n\n... *truncated*"))






<img src="https://documentation.marine.copernicus.eu/IMG/ANTARCTIC_OMI_SI_extent.png" width="300" height="200"/>



# Antarctic Sea Ice Extent from Reanalysis


**Product ID:** ANTARCTIC_OMI_SI_extent

**Description:** **DEFINITION**

Estimates of Antarctic sea ice extent are obtained from the surface of oceans grid cells that have at least 15% sea ice concentration. These values are cumulated in the entire Southern Hemisphere (excluding ice lakes) and from 1993 up to real time aiming to:
i) obtain the Antarctic sea ice extent as expressed in millions of km squared (106 km2) to monitor both the large-scale variability and mean state and change.
ii) to monitor the change in sea ice extent as expressed in millions of km squared per decade (106 km2/decade), or in sea ice extent loss/gain since the beginning of the time series as expressed in percent per decade (%/decade; reference period being the first date of the key figure b) dot-dashed trend line, Vaughan et al., 2013)). For the Southern Hemisphere, these trends are calculated from the annual mean values.
The Antarctic sea ice extent used here is based on the “multi-product” (GLOBAL_MULTIYEAR_PHY_ENS_001_031) approach as introduced in the second issue of the Ocean State Report (CMEMS OSR, 2017). Five global products have been used to build the ensemble mean, and its associated ensemble spread.

**CONTEXT**

Sea ice is frozen seawater that floats on the ocean surface. This large blanket of millions of square kilometers insulates the relatively warm ocean waters from the cold polar atmosphere. The seasonal cycle of the sea ice, forming and melting with the polar seasons, impacts both human activities and biological habitat. Knowing how and how much the sea ice cover is changing is essential for monitoring the health of the Earth as sea ice is one of the highest sensitive natural environments. Variations in sea ice cover can induce changes in ocean stratification and modify the key rule played by the cold poles in the Earth engine (IPCC, 2019).  
The sea ice cover is monitored here in terms of sea ice extent quantity. More details and full scientific evaluations can be found in the CMEMS Ocean State Report (Samuelsen et al., 2016; Samuelsen et al., 2018).
 
**CMEMS KEY FINDINGS**

With quasi regular highs and lows, the annual Antarctic sea ice extent shows large variability until several monthly record high in 2014 and record lows in 2017, 2018 and 2023. Since the year 1993, the Southern Hemisphere annual sea ice extent regularly alternates positive and negative trend. The period 1993-2023 have seen a slight decrease at a rate of -0.28*106km2 per decade. This represents a loss amount of -2.4% per decade of Southern Hemisphere sea ice extent during this period; with however large uncertainties. The last quarter of the year 2016 and years 2017 and 2018 experienced unusual losses of ice. The year 2023 is an exceptional year and its average has a strong impact on the whole time series.

Note: The key findings will be updated annually in November, in line with OMI evolutions.

**DOI (product):** 
https://doi.org/10.48670/moi-00186

**References:**

* IPCC Special Report on the Ocean and Cryosphere in a Changing Climate. (2019). In H. O. Pörtner, D. C. Roberts, V. Masson-Delmotte, P. Zhai, M. Tignor, E. Poloczanska, K. Mintenbeck, A. Alegría, M. Nicolai, A. Okem, J. Petzold, B. Rama, & N. M. Weyer (Eds.), IPCC Intergovernmental Panel on Climate Change: Geneva, Switzerland. https://www.ipcc.ch/srocc/
* Samuelsen et al., 2016: Sea Ice In: The Copernicus Marine Environment Monitoring Service Ocean State Report, issue 1, Journal of Operational Oceanography, 9, 2016, http://dx.doi.org/10.1080/1755876X.2016.1273446.
* Samuelsen et al., 2018: Sea Ice. In: The Copernicus Marine Environment Monitoring Service Ocean State Report, issue 2, Journal of Operational Oceanography, 11:sup1, 2018, DOI: 10.1080/1755876X.2018.1489208.
* Vaughan, D.G., J.C. Comiso, I. Allison, J. Carrasco, G. Kaser, R. Kwok, P. Mote, T. Murray, F. Paul, J. Ren, E. Rignot, O. Solomina, K. Steffen and T. Zhang, 2013: Observations: Cryosphere. In: Climate Change 2013: The Physical Science Basis. Contribution of Working Group I to the Fifth Assessment Report of the Intergovernmental Panel on Climate Change [Stocker, T.F., D. Qin, G.-K. Plattner, M.Tignor, S.K. Allen, J. Boschung, A. Nauels, Y. Xia, V. Bex and P.M. Midgley (eds.)]. Cambridge University Press, Cambridge, United Kingdom and New York, NY, USA, pp. 317–382, doi:10.1017/CBO9781107415324.012.





<img src="https://documentation.marine.copernicus.eu/IMG/ANTARCTIC_OMI_SI_extent_obs.png" width="300" height="200"/>



# Antarctic Monthly Sea Ice Extent from Observations Reprocessing


**Product ID:** ANTARCTIC_OMI_SI_extent_obs

**Description:** **DEFINITION**

Sea Ice Extent (SIE) is defined as the area covered by sufficient sea ice, that is the area of ocean having more than 15% Sea Ice Concentration (SIC). SIC is the fractional area of ocean surface that is covered with sea ice. SIC is computed from Passive Microwave satellite observations since 1979. 
SIE is often reported with units of 106 km2 (millions square kilometers). The change in sea ice extent (trend) is expressed in millions of km squared per decade (106 km2/decade). In addition, trends are expressed relative to the 1979-2022 period in % per decade.
These trends are calculated (i) from the annual mean values; (ii) from the September values (winter ice loss); (iii) from February values (summer ice loss). The annual mean trend is reported on the key figure, the September (maximum extent) and February (minimum extent) values are reported in the text below.
SIE includes all sea ice, except for lake and river ice.
See also section 1.7 in Samuelsen et al. (2016) for an introduction to this Ocean Monitoring Indicator (OMI).

**CONTEXT**

Sea ice is frozen seawater that floats at the ocean surface. This large blanket of millions of square kilometers insulates the relatively warm ocean waters from the cold polar atmosphere. The seasonal cycle of sea ice, forming and melting with the polar seasons, impacts both human activities and biological habitat. Knowing how and by how much the sea-ice cover is changing is essential for monitoring the health of the Earth (Meredith et al. 2019). 

**CMEMS KEY FINDINGS**

Since 1979, there has been an overall slight increase of sea ice extent in the Southern Hemisphere but a sharp decrease was observed after 2016. Over the period 1979-2022, the annual rate amounts to +0.02 +/- 0.05 106 km2 per decade (+0.18% per decade). Winter (September) sea ice extent trend amounts to +0.06 +/- 0.05106 km2 per decade (+0.32% per decade). Summer (February) sea ice extent trend amounts to -0.01+/- 0.05 106 km2 per decade (-0.38% per decade). These trend estimates are hardly significant, which is in agreement with the IPCC SROCC, which has assessed that ‘Antarctic sea ice extent overall has had no statistically significant trend (1979–2018) due to contrasting regional signals and large interannual variability (high confidence).’ (IPCC, 2019). Both June and July 2022 had the lowest average sea ice extent values for these months since 1979. 

**Figure caption**

a) The seasonal cycle of Southern Hemisphere sea ice extent expressed in millions of km2 averaged over the period 1979-2022 (red), shown together with the seasonal cycle in the year 2022 (green), and b) time series of yearly average Southern Hemisphere sea ice extent expressed in millions of km2. Time series are based on satellite observations (SMMR, SSM/I, SSMIS) by EUMETSAT OSI SAF Sea Ice Index (v2.2) with R&D input from ESA CCI. Details on the product are given in the corresponding PUM for this OMI. The change of sea ice extent over the period 1979-2022      is expressed as a trend in millions of square kilometers per decade and is plotted with a dashed line on panel b).

**DOI (product):** 
https://doi.org/10.48670/moi-00187

**References:**

* Meredith, M., M. Sommerkorn, S. Cassotta, C. Derksen, A. Ekaykin, A. Hollowed, G. Kofinas, A. Mackintosh, J. Melbourne-Thomas, M.M.C. Muelbert, G. Ottersen, H. Pritchard, and E.A.G. Schuur, 2019: Polar Regions. In: IPCC Special Report on the Ocean and Cryosphere in a Changing Climate [H.-O. Pörtner, D.C. Roberts, V. Masson-Delmotte, P. Zhai, M. Tignor, E. Poloczanska, K. Mintenbeck, A. Alegría, M. Nicolai, A. Okem, J. Petzold, B. Rama, N.M. Weyer (eds.)]. In press.
* IPCC, 2019: IPCC Special Report on the Ocean and Cryosphere in a Changing Climate [H.-O. Pörtner, D.C. Roberts, V. Masson-Delmotte, P. Zhai, M. Tignor, E. Poloczanska, K. Mintenbeck, A. Alegría, M. Nicolai, A. Okem, J. Petzold, B. Rama, N.M. Weyer (eds.)]. In press.
* Samuelsen, A., L.-A. Breivik, R.P. Raj, G. Garric, L. Axell, E. Olason (2016): Sea Ice. In: The Copernicus Marine Service Ocean State Report, issue 1, Journal of Operational Oceanography, 9:sup2, s235-s320, DOI: 10.1080/1755876X.2016.1273446





<img src="https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/ARCTIC_ANALYSISFORECAST_BGC_002_004.jpg" width="300" height="200"/>



# Arctic Ocean Biogeochemistry Analysis and Forecast


**Product ID:** ARCTIC_ANALYSISFORECAST_BGC_002_004

**Description:** The operational TOPAZ5-ECOSMO Arctic Ocean system uses the ECOSMO biological model coupled online to the TOPAZ5 physical model (ARCTIC_ANALYSISFORECAST_PHY_002_001 product). It is run daily to provide 10 days of forecast of 3D biogeochemical variables ocean. The coupling is done by the FABM framework.

Coupling to a biological ocean model provides a description of the evolution of basic biogeochemical variables. The output consists of daily mean fields interpolated onto a standard grid and 40 fixed levels in NetCDF4 CF format. Variables include 3D fields of nutrients (nitrate, phosphate, silicate), phytoplankton and zooplankton biomass, oxygen, chlorophyll, primary productivity, carbon cycle variables (pH, dissolved inorganic carbon and surface partial CO2 pressure in seawater) and light attenuation coefficient. Surface Chlorophyll-a from satellite ocean colour is assimilated every week and projected downwards using a modified Ardyna et al. (2013) method. A new 10-day forecast is produced daily using the previous day's forecast and the most up-to-date prognostic forcing fields.


... *truncated*

## Download data

In [74]:
import datetime as dt
# Product example: ARCTIC_ANALYSISFORECAST_BGC_002_004
DATASET_ID = "cmems_mod_arc_bgc_anfc_ecosmo_P1D-m"  #  ⟵ from product page (Select one of you choice)

# Small AOI covering Iceland and nearby Arctic (adjust as you wish)
bbox = dict(
    minimum_longitude=-40.0, maximum_longitude=40.0,
    minimum_latitude=60.0,  maximum_latitude=85.0
)

# Use "latest" day that is surely produced (yesterday is safe for NRT)
end = dt.datetime.utcnow().date()
start = end - dt.timedelta(days=1)
time_sel = dict(
    start_datetime=start.isoformat(),
    end_datetime=end.isoformat()
)

# Name of the file we download
out_nc = "arctic_bgc_chl.nc"

In [75]:
# We'll ask for "chl*" but stay robust: if variable name differs, we'll detect it later from the file itself.
# Request only surface layer (0 m). Many CMEMS BGC datasets use "depth" in meters.
cm.subset(
    dataset_id=DATASET_ID,
    variables=None,                      # keep None here; we’ll auto-pick "chl" variable after download
    **bbox,
    minimum_depth=0, maximum_depth=0,    # surface
    **time_sel,
    output_filename=out_nc
)

INFO - 2025-08-14T15:39:27Z - Selected dataset version: "202105"
INFO:copernicusmarine:Selected dataset version: "202105"
INFO - 2025-08-14T15:39:27Z - Selected dataset part: "default"
INFO:copernicusmarine:Selected dataset part: "default"
INFO - 2025-08-14T15:39:41Z - Starting download. Please wait...
INFO:copernicusmarine:Starting download. Please wait...


  0%|          | 0/54 [00:00<?, ?it/s]

INFO - 2025-08-14T15:40:15Z - Successfully downloaded to arctic_bgc_chl_(1).nc
INFO:copernicusmarine:Successfully downloaded to arctic_bgc_chl_(1).nc


ResponseSubset(file_path=PosixPath('arctic_bgc_chl_(1).nc'), output_directory=PosixPath('.'), filename='arctic_bgc_chl_(1).nc', file_size=42.86796183206107, data_transfer_size=531.1198778625953, variables=['chl', 'dissic', 'expc', 'kd', 'model_depth', 'no3', 'nppv', 'o2', 'ph', 'phyc', 'po4', 'si', 'spco2', 'zooc'], coordinates_extent=[GeographicalExtent(minimum=-39.999999999978115, maximum=39.92000000003438, unit='degrees_east', coordinate_id='longitude'), GeographicalExtent(minimum=60.0, maximum=85.0, unit='degrees_north', coordinate_id='latitude'), TimeExtent(minimum='2025-08-13T00:00:00+00:00', maximum='2025-08-14T00:00:00+00:00', unit='iso8601', coordinate_id='time'), GeographicalExtent(minimum=0.0, maximum=0.0, unit='m', coordinate_id='depth')], status='000', message='The request was successful.', file_status='DOWNLOADED')

## Convert the data to TIFF (to plot on map)

In [76]:
import xarray as xr
import rioxarray as rxr

ds = xr.open_dataset(out_nc, decode_times=True)

# ---- Find a chlorophyll-like variable (robust to name variations)
candidates = [v for v in ds.data_vars if ("chl" in v.lower()) or ("chlor" in v.lower())]
if not candidates:
    raise ValueError(f"No chlorophyll-like variable found in {list(ds.data_vars)}")
var = candidates[0]
da = ds[var]

# ---- Select a single time slice (nearest to end date)
if "time" in da.dims:
    da = da.sel(time=da["time"].max())

# ---- Some CMEMS grids are polar stereographic with x/y & grid_mapping.
# Try to attach CRS using the CF grid mapping, otherwise assume lat/lon.
def write_geotiff_from_da(da, tif_path="arctic_bgc_chl.tif"):
    # If we have lat/lon coords, reproject on-the-fly by assigning EPSG:4326
    if set(["lat", "latitude"]).intersection(da.coords) and set(["lon", "longitude"]).intersection(da.coords):
        # Normalize coordinate names
        lat_name = "lat" if "lat" in da.coords else "latitude"
        lon_name = "lon" if "lon" in da.coords else "longitude"
        da = da.rename({lat_name: "lat", lon_name: "lon"})
        # rioxarray expects y,x order named differently; use .rio.set_spatial_dims
        da = da.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=False)
        da = da.rio.write_crs(4326, inplace=False)
        da.rio.to_raster(tif_path)
        return tif_path

    # Else assume projected x/y with CF grid_mapping
    # rioxarray can parse CRS from the grid mapping variable when present.
    # Find 'x','y' or 'rlon','rlat' etc., normalize to x/y.
    spatial_dims = [d for d in da.dims if d.lower() in ("x","y","rlon","rlat","cols","rows")]
    if len(spatial_dims) >= 2:
        # Try to infer x/y order
        # Prefer something like ('x','y'); else sort by "x-like" first
        order = []
        for cand in ("x","rlon","cols"):
            if cand in da.dims: order.append(cand)
        for cand in ("y","rlat","rows"):
            if cand in da.dims: order.append(cand)
        # Fallback: keep existing
        if len(order) != 2: order = da.dims[:2]
        da = da.rename({order[0]: "x", order[1]: "y"})
        # Attach CRS from grid mapping if available
        gm = da.rio.grid_mapping
        if gm is None:
            # As a last resort, try common polar stereographic for ARC MFC
            # (Best effort; proper CRS comes from CF metadata.)
            da = da.rio.write_crs("EPSG:3857", inplace=False)  # Arctic Polar Stereographic
        # Ensure spatial dims are set
        da = da.rio.set_spatial_dims(x_dim="x", y_dim="y", inplace=False)
        da.rio.to_raster(tif_path)
        return tif_path

    raise RuntimeError("Could not determine geospatial coordinates/CRS to write GeoTIFF.")

tif_path = write_geotiff_from_da(da, "arctic_bgc_chl.tif")
tif_path

'arctic_bgc_chl.tif'

## Display the data on a map

In [ ]:
import leafmap

# Initialize map
m = leafmap.Map(center=[70, -20], zoom=3)

# Add background layer
m.add_basemap("Esri.OceanBasemap")

# Add our downloaded raster
m.add_raster(
    tif_path,
    layer_name="Chlorophyll-a (surface)",
    zoom_to_layer=True,
    colormap="viridis",
    vmin=0, # Min value for color scale
    vmax=10 # Max value for color scale
)

# Display map
m

## Display data directly from the web with no download

In [ ]:
import leafmap

# The layer identifier you can find them all here: https://data.marine.copernicus.eu/products
LAYER_ID = ("ARCTIC_ANALYSISFORECAST_BGC_002_004/"
            "cmems_mod_arc_bgc_anfc_ecosmo_P1D-m_202105/chl")

# Pick any of the listed times;
ISO_TIME = "2025-07-01T00:00:00Z"

# Optional: elevation (depth, in meters). The default in the XML is ~ -0.494 m (surface layer in z*)
ELEVATION = "-0.4940253794193268"   # or omit the &ELEVATION param to use the default

# Choose a TileMatrixSet that matches your basemap (Web Mercator or WGS84)
TMS = "EPSG:3857"

# Server-side styling. If viridis isn’t available on this layer, switch to 'cmap:matter' or use 'logScale' variant.
STYLE = "cmap:viridis"      # fallback: "cmap:matter" or "cmap:matter,logScale"

# Make URL
WMTS_URL = (
    "https://wmts.marine.copernicus.eu/teroWmts"
    "?SERVICE=WMTS&REQUEST=GetTile&VERSION=1.0.0"
    f"&LAYER={LAYER_ID}"
    "&FORMAT=image/png"
    f"&TILEMATRIXSET={TMS}"
    "&TILEMATRIX={z}&TILEROW={y}&TILECOL={x}"
    f"&time={ISO_TIME}"
    f"&STYLE={STYLE}"
    f"&ELEVATION={ELEVATION}"
)

# Prepare map
m = leafmap.Map(center=[30, 0], zoom=2)

# Add our WMTS layer
m.add_tile_layer(
    url=WMTS_URL,
    name="CMEMS Zooplankton (monthly)",
    attribution="Copernicus Marine"
)
m

The difference is that on the online data served as Web Map Tile Server (WMTS) it is not possible to process the data, while on the downloaded data it is possibe to combine it with other data or extract the single pixel values.